In [2]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)
import numpy as np
import config

import plotting_helpers as plots
import utility_functions as utils
from widget_helpers import dim_select, csd_select, kcsd_select, accordion

Populating the interactive namespace from numpy and matplotlib


KeyError: 'type'

In [ ]:
#Select dimensions of your setup
dim_select

In [ ]:
# Select a type of CSD source
csd_select

In [ ]:
csd_at, true_csd = utils.generate_csd(config.csd_profile, seed=2)
plots.show_csd(csd_at, true_csd)

In [ ]:
 #Lets add some electrodes which span the CSD space                                                                                                                                                                                                    
num_ele, ele_pos = utils.generate_electrodes()
plots.show_csd(csd_at, true_csd, show_ele=ele_pos)
print('Shape of the ele_pos array: ', ele_pos.shape)
print('Number of electrodes: ',ele_pos.shape[0])

In [ ]:
# Lets emulate the case when some of the electrodes are broken / used for stimulation
#ele_seed = np.random.RandomState(13)
#ele_fail = ele_seed.choice(num_ele, 3, replace=False)
#ele_pos_new = np.delete(ele_pos, ele_fail, axis=0)
#plots.show_csd(csd_at, true_csd, show_ele=ele_pos)
#print('Shape of the ele_pos array: ', ele_pos.shape)
#print('Number of electrodes: ',ele_pos.shape[0])

In [ ]:
# Lets compute the potential recorded by these electrodes due to the CSD
h = 10.
sigma = 1.
pots = utils.calculate_potential(csd_at, true_csd, ele_pos, h, sigma)
plots.show_pot(ele_pos, pots)

In [ ]:
#Select kCSD method
kcsd_select

In [ ]:
####TODO temporal illustration here
#Lets do KCSD!
k = config.kCSD(ele_pos, pots) 
est_csd = k.values()[:,...,0]
est_pos = k.estm_pos
plots.show_csd(est_pos, est_csd, show_ele=ele_pos, show_kcsd=True)

In [ ]:
k.cross_validate(Rs=np.arange(0.10, 0.25, 0.01), lambdas=np.logspace(15,-25, 35))
est_csd = k.values()[:,...,0]
plots.show_csd(est_pos, est_csd, show_ele=ele_pos, show_kcsd=True)